In [1]:
#!pip install auto-sklearn wandb

Random State for reproduceability

In [2]:
dagstuhl_seed=23372

# Data

In [3]:
import numpy as np
import pandas as pd

In [4]:
np.random.seed(dagstuhl_seed)

In [5]:
df = pd.read_csv('../../data/preprocessed.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.replace(['inactive', 'active'], [0,1]) # swap -1 (inactive) and 0 (unknown)

df_train = pd.read_csv('../../data/preprocessed-train.csv')
df_train = df_train.loc[:, ~df_train.columns.str.contains('^Unnamed')]
df_train = df_train.replace(['inactive', 'active'], [0,1]) # swap -1 (inactive) and 0 (unknown)

df_test = pd.read_csv('../../data/preprocessed-test.csv')
df_test = df_test.loc[:, ~df_test.columns.str.contains('^Unnamed')]
df_test = df_test.replace(['inactive', 'active'], [0,1]) # swap -1 (inactive) and 0 (unknown)

df.tail()

,label,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
4306,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4307,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4308,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4309,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4310,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
y_train = df_train['label']
X_train = df_train.loc[:, df_train.columns != "label"]

y_test = df_train['label']
X_test = df_train.loc[:, df_train.columns != "label"]

X_train.to_numpy().shape

(3592, 1024)

# AutoML Libraries

In [7]:
time_min = 30
time_sec = time_min * 60

## Auto-Sklearn2

In [8]:
from autosklearn.experimental.askl2 import AutoSklearn2Classifier
from sklearn.metrics import accuracy_score

In [9]:
logging_config = {
    "version": 1,
    "disable_existing_loggers": True,
    "formatters": {
        "custom": {
            # More format options are available in the official
            # `documentation <https://docs.python.org/3/howto/logging-cookbook.html>`_
            "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
        }
    },
    # Any DEBUG level msg will be printed to the console
    "handlers": {
        "console": {
            "level": "DEBUG",
            "formatter": "custom",
            "class": "logging.StreamHandler",
            "stream": "ext://sys.stdout",
        },
    },
    "loggers": {
        "": {  # root logger
            "level": "DEBUG",
        },
        "Client-EnsembleBuilder": {
            "level": "DEBUG",
            "handlers": ["console"],
        },
    },
}

In [10]:
%%time
automl = AutoSklearn2Classifier(
    time_left_for_this_task=time_sec,
    n_jobs=-1,
    delete_tmp_folder_after_terminate=False,
    logging_config=logging_config,
    tmp_folder="./autosk2-models",
)
automl.fit(X_train, y_train)

/opt/conda/lib/python3.9/site-packages/autosklearn/experimental/selector.py:26: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col, series in prediction.iteritems():
Capping the per_run_time_limit to 899.0 to have time for a least 2 models in each process.


2023-09-23 08:01:43,283 - Client-EnsembleBuilder - DEBUG - Restricting your function to 3072 mb memory.
2023-09-23 08:01:43,287 - Client-EnsembleBuilder - INFO - <Future: pending, key: fit_and_return_ensemble-78f4aceaad1a0badc99a1cbdad83e6b4>/<Client: 'inproc://172.30.0.2/13378/1' processes=8 threads=8> Started Ensemble builder job at 2023.09.23-08.01.43 for iteration 0.
2023-09-23 08:01:43,288 - Client-EnsembleBuilder - DEBUG - Restricting your function to 1785 seconds wall time.
2023-09-23 08:01:43,293 - Client-EnsembleBuilder - DEBUG - Allowing a grace period of 0 seconds.
2023-09-23 08:01:43,294 - Client-EnsembleBuilder - DEBUG - Function called with argument: (1790.8819329738617, 0), {}


No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1


2023-09-23 08:01:45,917 - Client-EnsembleBuilder - DEBUG - iteration=1 @ elapsed_time=10.484835386276245 has history=[{'Timestamp': Timestamp('2023-09-23 08:01:43.372112'), 'ensemble_optimization_score': 0.9276169265033407}]
2023-09-23 08:01:45,930 - Client-EnsembleBuilder - INFO - <Future: pending, key: fit_and_return_ensemble-291e1e973c10a19b22703c7a87ddfc2e>/<Client: 'inproc://172.30.0.2/13378/1' processes=8 threads=8> Started Ensemble builder job at 2023.09.23-08.01.45 for iteration 1.
2023-09-23 08:01:45,997 - Client-EnsembleBuilder - DEBUG - Restricting your function to 3072 mb memory.
2023-09-23 08:01:46,009 - Client-EnsembleBuilder - DEBUG - Restricting your function to 1783 seconds wall time.
2023-09-23 08:01:46,012 - Client-EnsembleBuilder - DEBUG - Allowing a grace period of 0 seconds.
2023-09-23 08:01:46,015 - Client-EnsembleBuilder - DEBUG - Function called with argument: (1788.168620109558, 1), {}


No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1


...

No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1


2023-09-23 08:03:00,396 - Client-EnsembleBuilder - DEBUG - iteration=17 @ elapsed_time=84.95851492881775 has history=[]
2023-09-23 08:03:00,415 - Client-EnsembleBuilder - INFO - <Future: pending, key: fit_and_return_ensemble-d466f7e279a3ca0f827a12b78ef845aa>/<Client: 'inproc://172.30.0.2/13378/1' processes=8 threads=8> Started Ensemble builder job at 2023.09.23-08.03.00 for iteration 17.
2023-09-23 08:03:00,463 - Client-EnsembleBuilder - DEBUG - Restricting your function to 3072 mb memory.
2023-09-23 08:03:00,484 - Client-EnsembleBuilder - DEBUG - Restricting your function to 1708 seconds wall time.
2023-09-23 08:03:00,486 - Client-EnsembleBuilder - DEBUG - Allowing a grace period of 0 seconds.
2023-09-23 08:03:00,489 - Client-EnsembleBuilder - DEBUG - Function called with argument: (1713.7055404186249, 17), {}


No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1


2023-09-23 08:03:15,377 - Client-EnsembleBuilder - DEBUG - iteration=18 @ elapsed_time=99.93538665771484 has history=[]
2023-09-23 08:03:15,427 - Client-EnsembleBuilder - INFO - <Future: pending, key: fit_and_return_ensemble-2388e6f0296b4c0efdacc7ec104b5e36>/<Client: 'inproc://172.30.0.2/13378/1' processes=8 threads=8> Started Ensemble builder job at 2023.09.23-08.03.15 for iteration 18.
2023-09-23 08:03:15,643 - Client-EnsembleBuilder - DEBUG - Restricting your function to 3072 mb memory.
2023-09-23 08:03:15,649 - Client-EnsembleBuilder - DEBUG - Restricting your function to 1693 seconds wall time.
2023-09-23 08:03:15,652 - Client-EnsembleBuilder - DEBUG - Allowing a grace period of 0 seconds.
2023-09-23 08:03:15,655 - Client-EnsembleBuilder - DEBUG - Function called with argument: (1698.5452325344086, 18), {}
2023-09-23 08:03:25,132 - Client-EnsembleBuilder - DEBUG - iteration=19 @ elapsed_time=109.67833161354065 has history=[{'Timestamp': Timestamp('2023-09-23 08:03:15.924634'), 'en

AutoSklearn2Classifier(delete_tmp_folder_after_terminate=False,
                       logging_config={'disable_existing_loggers': True,
                                       'formatters': {'custom': {'format': '%(asctime)s '
                                                                           '- '
                                                                           '%(name)s '
                                                                           '- '
                                                                           '%(levelname)s '
                                                                           '- '
                                                                           '%(message)s'}},
                                       'handlers': {'console': {'class': 'logging.StreamHandler',
                                                                'formatter': 'custom',
                                                                'level': 'DEB

In [11]:
predictions = automl.predict(X_test)
print("Accuracy score:", accuracy_score(y_test, predictions))

Accuracy score: 0.9306792873051225


## Auto-Sklearn1

In [12]:
from autosklearn.classification import AutoSklearnClassifier

In [13]:
%%time
automl = AutoSklearnClassifier(
    time_left_for_this_task=time_sec,
    n_jobs=-1,
    delete_tmp_folder_after_terminate=False,
    logging_config=logging_config,
    tmp_folder="./autosk1-models",
)
automl.fit(X_train, y_train)

Capping the per_run_time_limit to 899.0 to have time for a least 2 models in each process.


2023-09-23 08:31:54,110 - Client-EnsembleBuilder - DEBUG - Restricting your function to 3072 mb memory.
2023-09-23 08:31:54,119 - Client-EnsembleBuilder - DEBUG - Restricting your function to 1780 seconds wall time.
2023-09-23 08:31:54,110 - Client-EnsembleBuilder - INFO - <Future: pending, key: fit_and_return_ensemble-c255886a9fb5adac742558d5afa97c6f>/<Client: 'inproc://172.30.0.2/13378/39' processes=8 threads=8> Started Ensemble builder job at 2023.09.23-08.31.54 for iteration 0.
2023-09-23 08:31:54,132 - Client-EnsembleBuilder - DEBUG - Allowing a grace period of 0 seconds.
2023-09-23 08:31:54,150 - Client-EnsembleBuilder - DEBUG - Function called with argument: (1785.578623533249, 0), {}


No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1


2023-09-23 08:31:56,476 - Client-EnsembleBuilder - DEBUG - iteration=1 @ elapsed_time=16.267464876174927 has history=[{'Timestamp': Timestamp('2023-09-23 08:31:54.239873'), 'ensemble_optimization_score': 0.927487352445194}]
2023-09-23 08:31:56,505 - Client-EnsembleBuilder - DEBUG - Restricting your function to 3072 mb memory.
2023-09-23 08:31:56,506 - Client-EnsembleBuilder - INFO - <Future: pending, key: fit_and_return_ensemble-6f142f1efe5a0b9795931747f925e2e4>/<Client: 'inproc://172.30.0.2/13378/39' processes=8 threads=8> Started Ensemble builder job at 2023.09.23-08.31.56 for iteration 1.
2023-09-23 08:31:56,509 - Client-EnsembleBuilder - DEBUG - Restricting your function to 1778 seconds wall time.
2023-09-23 08:31:56,546 - Client-EnsembleBuilder - DEBUG - Allowing a grace period of 0 seconds.
2023-09-23 08:31:56,554 - Client-EnsembleBuilder - DEBUG - Function called with argument: (1783.183453798294, 1), {}
2023-09-23 08:32:05,037 - Client-EnsembleBuilder - DEBUG - iteration=2 @ el

AutoSklearnClassifier(delete_tmp_folder_after_terminate=False,
                      ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                      logging_config={'disable_existing_loggers': True,
                                      'formatters': {'custom': {'format': '%(asctime)s '
                                                                          '- '
                                                                          '%(name)s '
                                                                          '- '
                                                                          '%(levelname)s '
                                                                          '- '
                                                                          '%(message)s'}},
                                      'handlers': {'console': {'class': 'logging.StreamHandler',
                                                               'for

In [14]:
predictions = automl.predict(X_test)
print("Accuracy score:", accuracy_score(y_test, predictions))

Accuracy score: 0.9535077951002228


In [15]:
# View the models found by auto-sklearn
print(automl.leaderboard()) 

          rank  ensemble_weight                type      cost   duration
model_id                                                                
169          1             0.32       random_forest  0.063238   8.582371
71           2             0.02       random_forest  0.065767  16.407669
153          3             0.14       random_forest  0.065767  20.411429
41           4             0.02       random_forest  0.068297  44.508180
47           5             0.02       random_forest  0.068297  32.901064
124          6             0.02  passive_aggressive  0.069140  52.584608
8            7             0.06         extra_trees  0.069983  22.725614
115          8             0.02         extra_trees  0.070826  37.811748
31           9             0.02       random_forest  0.071669  34.589401
94          10             0.02            adaboost  0.071669  73.788178
133         11             0.34            adaboost  0.071669   8.525835


In [16]:
from pprint import pprint
# Print the final ensemble constructed by auto-sklearn
pprint(automl.show_models(), indent=4)

{   8: {   'balancing': Balancing(random_state=1),
           'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7f99cdb01d60>,
           'cost': 0.0699831365935919,
           'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f99cd6fba00>,
           'ensemble_weight': 0.06,
           'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f99cdb01dc0>,
           'model_id': 8,
           'rank': 1,
           'sklearn_classifier': ExtraTreesClassifier(criterion='entropy', max_features=65, min_samples_split=3,
                     n_estimators=512, n_jobs=1, random_state=1,
                     warm_start=True)},
    31: {   'balancing': Balancing(random_state=1),
            'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7f99cd2e36a0>,
            'cost': 0.07166947723440131,
   